In [2]:
from hyperopt import hp
from hyperopt import fmin, tpe
import tensorflow as tf
import pandas as pd
import pickle

import sys, os
sys.path.append(os.path.abspath('.\\..\\'))

from model.wen_keyes.wen_keyes_params import WenKeyesParams
from model.wen_keyes.wen_keyes import WenKeyes
from model.loss_functions.dice_loss import dice_loss
from model.loss_functions.tversky_loss import tversky_loss
from session.cli_parser import CLIParser
from session.session import Session

In [7]:
df = pd.read_csv(r'C:\Users\ricof\Documents\AI-Cup 2022\logs\csv-03222022.log')

In [6]:
A = {
    'a' : [1, 2, 3],
    'b' : [0.5, 0.7, 0.5]
}

B = {
    'a' : [2, 2, 1],
    'b' : [0.5, 0.5, 0.3]
}

C = {
    'a' : [2, 3, 3],
    'b' : [0.5, 0.6, 0.2]
}

In [11]:
D = C
D['a'].index(max(D['a']))

1

In [20]:
def avg_over_histories(hists, target: str, cols = []):
    avg_dict = {}
    for key in [target, *cols]:
        avg_dict[key] = []
    for h in hists:
        max_index = h[target].index(max(h[target]))
        avg_dict[target].append((h[target])[max_index])
        for c in cols:
            avg_dict[c].append((h[c])[max_index])
    for key in avg_dict.keys():
        avg_dict[key] = sum(avg_dict[key]) / len(avg_dict[key])
    return avg_dict

In [22]:
l = ['loss', 'binary_crossentropy', 
                'dice_loss', 
                'accuracy', 
                'auc', 
                'precision', 
                'recall', 
                'precision_at_recall', 
                'mean_absolute_error', 
                'val_loss', 
                'val_binary_crossentropy', 
                'val_dice_loss', 
                'val_accuracy', 
                'val_auc', 
                'val_precision', 
                'val_recall', 
                'val_precision_at_recall', 
                'val_mean_absolute_error']

In [3]:
def get_partial_dict(dictionary: dict, keys: list):
    partial_dict = {}
    for key in keys:
        partial_dict[key] = dictionary[key]
    return partial_dict

In [4]:
parser = CLIParser()
cli_args = ['-id', '123456789013', 
            '-l', 'tversky_loss', '-lp', 'beta', '0.4', 
            '-lr', '0.001', '-e', '1', '-o', 'Adam', 
            '-mc', 'WenKeyes', '-mp', 'default',
            '-tg', 'default', '-vg', 'default',
            '-tts', '0.75', '-aug', 'default'
           ]
cli_args = parser.parse_args(cli_args)
model_params = WenKeyesParams.get_updated(WenKeyesParams.default, **{'base_filter_count' : 32})

In [24]:
for i in range(16, 17):
    print(i)
    model_params = WenKeyesParams.get_updated(WenKeyesParams.default, **{'base_filter_count' : i, 'block_count' : 13})
    model = WenKeyes(**model_params).get_model()

16


ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concatenation axis. Received: input_shape=[(None, 4, 512), (None, 1, 512)]

In [12]:
i = 128
model_params = WenKeyesParams.get_updated(WenKeyesParams.default, **{'base_filter_count' : i, 'block_count' : 9})
model = WenKeyes(**model_params).get_model()

In [13]:
model.summary()

Model: "WEN_KEYES_U_Net"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input/S (InputLayer)           [(None, 1024, 16)]   0           []                               
                                                                                                  
 Conv1D/B1.LG0 (Conv1D)         (None, 1024, 128)    6272        ['Input/S[0][0]']                
                                                                                                  
 BatchNorm/B1.LG0 (BatchNormali  (None, 1024, 128)   512         ['Conv1D/B1.LG0[0][0]']          
 zation)                                                                                          
                                                                                                  
 Activation/B1.LG0 (Activation)  (None, 1024, 128)   0           ['BatchNorm/B1.LG0[

 UpSampling1D/B5 (UpSampling1D)  (None, 16, 2048)    0           ['Activation/B5.LG1[0][0]']      
                                                                                                  
 Conv1D/B5.L (Conv1D)           (None, 16, 1024)     6292480     ['UpSampling1D/B5[0][0]']        
                                                                                                  
 Concatenate/B6 (Concatenate)   (None, 16, 2048)     0           ['Conv1D/B5.L[0][0]',            
                                                                  'Activation/B4.LG1[0][0]']      
                                                                                                  
 Conv1D/B6.LG0 (Conv1D)         (None, 16, 1024)     6292480     ['Concatenate/B6[0][0]']         
                                                                                                  
 BatchNorm/B6.LG0 (BatchNormali  (None, 16, 1024)    4096        ['Conv1D/B6.LG0[0][0]']          
 zation)  

                                                                                                  
 Conv1D/BF (Conv1D)             (None, 1024, 2)      258         ['Activation/B9.LG1[0][0]']      
                                                                                                  
 Activation/BF (Activation)     (None, 1024, 2)      0           ['Conv1D/BF[0][0]']              
                                                                                                  
Total params: 49,170,946
Trainable params: 49,147,394
Non-trainable params: 23,552
__________________________________________________________________________________________________


In [3]:
space: dict = {
        'loss' : hp.choice('loss', [('dice_loss', 0), ('BinaryCrossentropy', 0), ('tversky_loss', 0.15), ('tversky_loss', 0.35), ('tversky_loss', 0.65), ('tversky_loss', 0.85)]),
        'base_filter_count' : hp.randint('base_filter_count.default', 2, 256), 
        'block_count' : hp.choice('block_count', [3, 5, 7, 9, 11, 13]),
        'kernel_size' : {'default' : hp.randint('kernel_size.default', 2, 8)}, 
        'dropout' : {'default' : hp.uniform('dropout.default', 0.0, 1.0)},
    }

In [4]:
parser = CLIParser()

In [5]:
cli_args = ['-id', '123456789012', 
            '-l', 'tversky_loss', '-lp', 'beta', '0.4', 
            '-lr', '0.001', '-e', '1', '-o', 'Adam', 
            '-mc', 'WenKeyes', '-mp', 'default',
            '-tg', 'default', '-vg', 'default',
            '-tts', '0.75', '-aug', 'default'
           ]
cli_args = parser.parse_args(cli_args)

In [6]:
def objective(args: dict) -> float:
    # print(args)
    # return 0.0

    model_dict = get_partial_dict(args, keys=['base_filter_count', 'block_count', 'dropout'])
    model_params = WenKeyesParams.get_updated(WenKeyesParams.default, **model_dict)
    
    loss_name, beta = args['loss']
    if loss_name == 'BinaryCrossentropy':
        loss = tf.keras.losses.BinaryCrossentropy()
    elif loss_name == 'dice_loss':
        loss = dice_loss
    elif loss_name == 'tversky_loss':
        loss = tversky_loss(beta) 
        
    session = Session(cli_args)
    session.model_params = model_params
    session.loss = loss
    
    history = session.execute()
    
    print(args)
    return 0.0
    
    history = session.execute()
    
    return -max(history.history['val_accuracy'])

In [7]:
best = fmin(objective, space, algo=tpe.suggest, max_evals=1)
print(best)

2022-04-15 00:37:58.946285   [Session] Starting run in current session ...                                             
2022-04-15 00:37:58.950271   [Session] Retrieving model ...                                                            
{'sequence_length': 1024, 'channels': 16, 'input_shape': (1024, 16), 'class_count': 2, 'concat_axis': 2, 'base_filter_count': 238, 'kernel_size': {'default': 3, '1.1': 3, '1.2': 3, '2.1': 3, '2.2': 3, '3.1': 3, '3.2': 3, '4.1': 3, '4.2': 3, '4.l': 3, '5.1': 3, '5.2': 3, '5.l': 3, '6.1': 3, '6.2': 3, '6.l': 3, '7.1': 3, '7.2': 3}, 'pool_size': {'default': 4, '1': 4, '2': 4, '3': 4}, 'strides': {'default': 1, '1.1': 1, '1.2': 1, '2.1': 1, '2.2': 1, '3.1': 1, '3.2': 1, '4.1': 1, '4.2': 1, '5.1': 1, '5.2': 1, '6.1': 1, '6.2': 1, '7.1': 1, '7.2': 1}, 'block_count': 7, 'dropout': {'default': 0.9338291440786234, '1': 0.9338291440786234, '2': 0.9338291440786234, '3': 0.9338291440786234, '4': 0.9338291440786234, '5': 0.9338291440786234, '6': 0.933829144078623

ERROR:root:Graph execution error:

Detected at node 'WEN_KEYES_U_Net/Conv1D/B1.LG0/Conv1D' defined at (most recent call last):
    File "c:\python39\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\python39\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\ricof\.virtualenvs\src-ga5UjBuD\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\Users\ricof\.virtualenvs\src-ga5UjBuD\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\ricof\.virtualenvs\src-ga5UjBuD\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "C:\Users\ricof\.virtualenvs\src-ga5UjBuD\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "c:\python39\lib\asyncio\base_events.py", line 596, in run_forev

2022-04-15 00:38:54.044455   [Session] Starting run in current session ...                                             
2022-04-15 00:38:54.051414   [Session] Retrieving model ...                                                            
{'sequence_length': 1024, 'channels': 16, 'input_shape': (1024, 16), 'class_count': 2, 'concat_axis': 2, 'base_filter_count': 238, 'kernel_size': {'default': 3, '1.1': 3, '1.2': 3, '2.1': 3, '2.2': 3, '3.1': 3, '3.2': 3, '4.1': 3, '4.2': 3, '4.l': 3, '5.1': 3, '5.2': 3, '5.l': 3, '6.1': 3, '6.2': 3, '6.l': 3, '7.1': 3, '7.2': 3}, 'pool_size': {'default': 4, '1': 4, '2': 4, '3': 4}, 'strides': {'default': 1, '1.1': 1, '1.2': 1, '2.1': 1, '2.2': 1, '3.1': 1, '3.2': 1, '4.1': 1, '4.2': 1, '5.1': 1, '5.2': 1, '6.1': 1, '6.2': 1, '7.1': 1, '7.2': 1}, 'block_count': 7, 'dropout': {'default': 0.9338291440786234, '1': 0.9338291440786234, '2': 0.9338291440786234, '3': 0.9338291440786234, '4': 0.9338291440786234, '5': 0.9338291440786234, '6': 0.933829144078623

INFO:tensorflow:Assets written to: C:\Users\ricof\Documents\AI-Cup 2022\models\123456789012\keras-model\assets


INFO:tensorflow:Assets written to: C:\Users\ricof\Documents\AI-Cup 2022\models\123456789012\keras-model\assets



2022-04-15 00:39:59.489876   [Session] Successfully saved model.                                                       
  0%|                                                                            | 0/1 [02:00<?, ?trial/s, best loss=?]

ERROR:hyperopt.fmin:job exception: local variable 'history' referenced before assignment



  0%|                                                                            | 0/1 [02:00<?, ?trial/s, best loss=?]


UnboundLocalError: local variable 'history' referenced before assignment

In [1]:
from ray import tune

dic = {
        'backbone_length': tune.randint(2, 7),
        'kernel_size': tune.randint(2, 11),
        'n_fold_convolutions' : tune.randint(1, 5),
        'multi_heads' : tune.randint(1, 5),
        'attention_key_dim' : tune.randint(4, 11),
        'inception_kernel_size' : tune.randint(3, 7)
}

In [7]:
for key in dic.keys():
    msg = f'''{key} = config[\'{key}\']\n
        if type({key}) == float:\n
            config[\'{key}\'] = int(config[\'{key}\'])\n
        elif type({key}) == int:\n
            pass\n
        else:\n            
            raise WrongDataTypeInSearchSpaceError(\n
                variable=\'{key}\',\n
                data_type=type(config[\'{key}\']),\n
                expected_types=[float, int]\n
            )\n\n'''
    print(msg)

backbone_length = config['backbone_length']

        if type(backbone_length) == float:

            config['backbone_length'] = int(config['backbone_length'])

        elif type(backbone_length) == int:

            pass

        else:
            
            raise WrongDataTypeInSearchSpaceError(

                variable='backbone_length',

                data_type=type(config['backbone_length']),

                expected_types=[float, int]

            )


kernel_size = config['kernel_size']

        if type(kernel_size) == float:

            config['kernel_size'] = int(config['kernel_size'])

        elif type(kernel_size) == int:

            pass

        else:
            
            raise WrongDataTypeInSearchSpaceError(

                variable='kernel_size',

                data_type=type(config['kernel_size']),

                expected_types=[float, int]

            )


n_fold_convolutions = config['n_fold_convolutions']

        if type(n_fold_convolutions) == flo